In [1]:
import pandas as pd
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torchmetrics import Accuracy, F1Score, Precision, Recall, Specificity
import torch
from sklearn.metrics import log_loss


In [2]:
def metrics(targets, predictions, number_of_classes, module_type='Train'):
    precision = Precision(task="multiclass", average='macro', num_classes=number_of_classes)
    recall = Recall(task="multiclass", average='macro', num_classes=number_of_classes)
    f_measure = F1Score(task="multiclass", average='macro', num_classes=number_of_classes)
    specificity = Specificity(task="multiclass", average='macro', num_classes=number_of_classes)
    accuracy = Accuracy(task="multiclass", average='macro', num_classes=number_of_classes)
    targets, predictions = torch.tensor(targets), torch.tensor(predictions)
    precision_score = precision(predictions, targets).numpy().reshape(-1)[0]
    recall_score = recall(predictions, targets).numpy().reshape(-1)[0]
    f_measure_score = f_measure(predictions, targets).numpy().reshape(-1)[0]
    specificity_score = specificity(predictions, targets).numpy().reshape(-1)[0]
    Accuracy_score = accuracy(predictions, targets).numpy().reshape(-1)[0]

    metric_results = {
        "Precision": precision_score,
        "recall": recall_score,
        "FMeasure": f_measure_score,
        "Specificity": specificity_score,
        "Accuracy": Accuracy_score,
    }
    return pd.DataFrame.from_dict(metric_results, orient="index", columns=[module_type])

In [3]:
# Load and Normalize data

dataset = datasets.load_iris(as_frame=True)['data']
target = datasets.load_iris(as_frame=True)['target'].values

X_train, X_test, y_train, y_test = train_test_split(
    dataset, target, test_size=0.2, random_state=1, shuffle=True)

number_of_classes = 3

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [4]:
LRModel = LogisticRegression(random_state=0).fit(X_train_scaled, y_train)

y_train_output = LRModel.predict(X_train_scaled)
y_test_output = LRModel.predict(X_test_scaled)

In [5]:
train_out_prob = LRModel.predict_proba(X_train_scaled)
test_out_prob = LRModel.predict_proba(X_test_scaled)
train_logloss = log_loss(y_train, train_out_prob)
test_logloss = log_loss(y_test, test_out_prob)
print(f'train log loss :{train_logloss},\ntest log loss: {test_logloss} ')

train log loss :0.1446540806101643,
test log loss: 0.15406257484007455 


In [6]:
module_type = ['Train', 'Test']
train_metric = metrics(y_train, y_train_output, number_of_classes, module_type=module_type[0])
test_metric = metrics(y_test, y_test_output, number_of_classes, module_type=module_type[1])

print(pd.concat([train_metric, test_metric], axis=1))

                Train      Test
Precision    0.975926  0.952381
recall       0.974406  0.974359
FMeasure     0.975065  0.961026
Specificity  0.987212  0.986111
Accuracy     0.974406  0.974359
